In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn
!pip install qwen_vl_utils av

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0i2s7yt3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0i2s7yt3
  Resolved https://github.com/huggingface/transformers to commit 919220dab1e29f4d04eacd61a197a45a4fec2613
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10328720 sha256=8099c93776df4bd35efb528dcc08bb90451d06ff7cfac41fcc86aed9a40f0af3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ys1vgmr0/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
  Created wheel for flash_attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc4

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [3]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [4]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
model_name, torch_dtype=torch.bfloat16, attn_implementation="eager", device_map="auto"
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [5]:
processor = AutoProcessor.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [6]:


messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/content/3197622-hd_1920_1080_25fps.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "HOw many doctors are there and what they are doing?."},
        ],
    }
]

In [7]:
messages

[{'role': 'user',
  'content': [{'type': 'video',
    'video': '/content/3197622-hd_1920_1080_25fps.mp4',
    'max_pixels': 151200,
    'fps': 1.0},
   {'type': 'text',
    'text': 'HOw many doctors are there and what they are doing?.'}]}]

In [8]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)


In [9]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

qwen-vl-utils using torchvision to read video.


In [10]:
inputs = inputs.to("cuda")


In [11]:
generated_ids = model.generate(**inputs, max_new_tokens=512)

In [12]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]


In [13]:

output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [14]:
print(output_text)

['There are four doctors in the image. They are standing in a surgical room, wearing medical scrubs and caps. They appear to be preparing for or discussing a medical procedure.']
